In [19]:
import boto3
import datetime

# Configurar MinIO:
minio_endpoint = "http://minio.heady:9000"
access_key = "minioadmin"
secret_key = "minioadmin"
bucket_name = "biera-datalake"

s3_client = boto3.client(
    "s3",
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key
)

In [20]:
# Obtém a data atual
today = datetime.datetime.now(datetime.UTC)
year = today.strftime("%Y")
month = today.strftime("%m")
day = today.strftime("%d")

In [21]:
import json
from io import BytesIO
import pandas as pd

# Mostrar todas as colunas
pd.set_option('display.max_columns', None)

# Mostrar todas as linhas
pd.set_option('display.max_rows', None)

# Mostrar todo o conteúdo de cada célula
pd.set_option('display.max_colwidth', None)

def get_files_for_stream(stream_name):
    """Lista arquivos de um determinado stream no formato correto."""
    filename_pattern = f"{year}_{month}_{day}.csv"
    prefix = f"meta-ads/bronze/{stream_name}/{filename_pattern}"
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    arquivos = []
    if "Contents" in response:
        arquivos = [obj["Key"] for obj in response["Contents"]]
        print(f"Arquivos encontrados para {stream_name}:")
        for arquivo in arquivos:
            print(f" - {arquivo}")
    else:
        print(f"Nenhum arquivo encontrado para {stream_name}.")
    
    return arquivos

def load_csv_from_minio(bucket, object_key):
    """Baixa um arquivo CSV do MinIO e carrega no Pandas DataFrame."""
    response = s3_client.get_object(Bucket=bucket, Key=object_key)
    csv_content = response['Body'].read()
    
    # Criar DataFrame
    return pd.read_csv(BytesIO(csv_content))


In [30]:
stream_name = "campaigns"
files = get_files_for_stream(stream_name)

Arquivos encontrados para campaigns:
 - meta-ads/bronze/campaigns/2025_03_15.csv


In [31]:
ad_creatives = files[0]

data = load_csv_from_minio(bucket_name, ad_creatives)

data.head()

,_airbyte_raw_id,_airbyte_extracted_at,_airbyte_meta,_airbyte_generation_id,id,name,status,adlabels,objective,spend_cap,stop_time,account_id,start_time,buying_type,issues_info,bid_strategy,created_time,daily_budget,updated_time,lifetime_budget,budget_remaining,effective_status,boosted_object_id,configured_status,source_campaign_id,special_ad_category,smart_promotion_type,budget_rebalance_flag,special_ad_category_country
0,0275ee69-5b76-4224-a14f-85d9e5ba91d2,1742054917337,"{""sync_id"":90,""changes"":[]}",12,120217962796120086,[LEADS] [IG] [PÚBLICO QUENTE] Formulário para contato,ACTIVE,NaN,OUTCOME_LEADS,NaN,NaN,596780979471140,2025-03-12T16:55:27+0000,AUCTION,NaN,LOWEST_COST_WITHOUT_CAP,2025-03-12T16:55:26+0000,100,2025-03-12T16:56:18+0000,NaN,66,ACTIVE,18007653137727155,ACTIVE,120217556107850080,NONE,GUIDED_CREATION,False,NaN
1,957f049a-b12b-4589-b186-972b38f9e59a,1742054917338,"{""sync_id"":90,""changes"":[]}",12,120217556107850086,[LEADS] [IG] [PÚBLICO FRIO] Formulário para contato,ACTIVE,NaN,OUTCOME_LEADS,NaN,NaN,596780979471140,2025-03-08T12:41:05+0000,AUCTION,NaN,LOWEST_COST_WITHOUT_CAP,2025-03-08T12:41:04+0000,150,2025-03-12T16:56:24+0000,NaN,53,ACTIVE,17899120638145445,ACTIVE,0,NONE,GUIDED_CREATION,False,NaN
